In [1]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage

The following code snippets try to check the correctness of the transformer parameters, defined by doi.org/10.1088/0953-2048/20/5/010

The turn voltage calculated in 2 different ways:
   - from the core parameters
   - from the given line/phase voltages and turn number of the windings

The given parameters of the transformer core are:
 - the core diameter is 396 mm, the core radius is 198 mm
 - the flux density in the core is 1.275 T
 - the filling factor of the core was not given, we calculating by a usual 88 %
 - the network frequency is 50 Hz
 
Using the turn voltage base function of the library:

In [2]:
ut = turn_voltage(1.275, 198.0, 0.88, 50.0)
print(round(ut,2), 'V')

30.68 V


Turn voltage can be calculated from the winding parameters of the star/star connected three phase transformer:

In [3]:
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V


Therefore, some parameter of the core is wrong, maybe the flux density of the core is a bit lower in the practice, in this case the flux density should be 0.963 T

In [4]:
ut = turn_voltage(0.963, 198.0, 0.88, 50.0)
print(ut)

23.17086180119901


In [21]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("630kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)

# FEM calculation
trafo_model.fem_simulation()


Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Elapsed time: 00:05.138 s
0.030553156925387605 0.021460858500410324


The following command can list the main results of the calculation:

In [22]:
print(trafo_model.results)

MainResults(wh=809.5, feasible=True, core_loss=2.409803543694024, load_loss=3.7932096082603035, copper_mass=0.0, sci=0.01671196819145312, window_width=139.33646524072992, core_mass=4982.1722538118, turn_voltage=30.677932291307098, capitalized_cost=0.0, fem_based_sci=2.278111264090742, fem_bax=0.030553156925387605, fem_brad=0.021460858500410324)


The resulting fem-based short circuit impedance is 2.28 %, which is very close to the given 2.45 %, however, the analytically calculated sci is only 1.7%. The reason is that the analytical formula contains the turn voltage, which is different, the core somehow not sized properly. 

In [19]:
print(trafo_model.hv_winding)

WindingDesign(inner_radius=290.54304750889895, thickness=13.586228900167132, winding_height=354.83, filling_factor=100.0, current_density=1.88, mass=240.46845854358773, dc_loss=2.0567863621010245, ac_loss=0.0, outer_radius=304.1292764090661)


In [23]:
print(trafo_model.lv_winding)

WindingDesign(inner_radius=244.0, thickness=6.074866471427201, winding_height=342.5, filling_factor=100.0, current_density=3.29, mass=86.22857482554885, dc_loss=2.25869905458152, ac_loss=0.0, outer_radius=250.0748664714272)
